In [ ]:
!pip install earthengine-api --upgrade

Requirement already up-to-date: earthengine-api in /usr/local/lib/python3.6/dist-packages (0.1.226)


In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
import numpy as np
import os

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tk0IzghxWUyFubL3SpxnkBihIeD6GMK0HLyr1aY6xCo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1wH8FtwGa5v1Nn5aeTtWHcpDVA5Q_KL6potKtffDjNS4dnSC6IvGVGc

Successfully saved authorization token.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
def getPolygon(lat_long_tup, scale_):
  lat = lat_long_tup[0]
  long_ = lat_long_tup[1]
  aoi = ee.Geometry.Polygon(
        [[[lat-(scale_/2), long_-(scale_/2)],
          [lat-(scale_/2), long_+(scale_/2)],
          [lat+(scale_/2), long_+(scale_/2)],
          [lat+(scale_/2), long_-(scale_/2)]]], None, False)
  return aoi

# only works for image products of TOA and SR:
def tifToPNG(img, img_product, aoi, filename='PNG1.png', scale=1, sclx=1, nums="123"):
  print("Processing conversion for", filename+"...")

  # Get a scaled 2-d pixel array for AOI 
  band_arrs = img.reproject(crs=img.projection().crs(), scale=sclx).sampleRectangle(region=aoi, defaultValue=0)
  
  # Get individual band arrays.
  band_arr_b4 = band_arrs.get('B'+nums[0])
  band_arr_b5 = band_arrs.get('B'+nums[1])
  band_arr_b6 = band_arrs.get('B'+nums[2])
  
  # Transfer the arrays from server to client and cast as np array.
  np_arr_b4 = np.array(band_arr_b4.getInfo())
  np_arr_b5 = np.array(band_arr_b5.getInfo())
  np_arr_b6 = np.array(band_arr_b6.getInfo())

  # Expand the dimensions of the images so they can be concatenated into 3-D.
  np_arr_b4 = np.expand_dims(np_arr_b4, 2)
  np_arr_b5 = np.expand_dims(np_arr_b5, 2)
  np_arr_b6 = np.expand_dims(np_arr_b6, 2)
  
  # Stack the individual bands to make a 3-D array.
  rgb_img = np.concatenate((np_arr_b6, np_arr_b5, np_arr_b4), 2)

  # Scale the data to [0, 255] to show as an RGB image.
  rgb_img_test = (255*((rgb_img - 100)/3500)).astype('uint8') if img_product == 'SR' else (255*(rgb_img/0.35)).astype('uint8')
  img_rescaled = rescale(rgb_img_test, scale=(scale, scale, 1))
  plt.imsave(filename, img_rescaled)

  return img_rescaled

Function usage demo:

In [ ]:
path = 'drive/My Drive/CV_DATA/'
coodnts = open(path+'coodnts.txt')

for line in coodnts.readlines():
  xtract = line.split(',')
  xtract = [xtract[0].strip(), xtract[1].strip(), xtract[2].strip()]
  filenameold = xtract[0]+"OLD.png"
  filenamenew = xtract[0]+"NEW.png"
  filename2010 = xtract[0]+"2010.png"

  aoi = getPolygon((float(xtract[2]), float(xtract[1])), 0.12) 
  
  # old version
  img = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate('1998-01', '2000-12-31').median().select(['B1', 'B2', 'B3'])
  tifToPNG(img, 'SR', aoi, filename=path+"old/"+filenameold, scale=1.5, sclx=30)

  # new version
  img = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate('2018-01', '2020-12-31').median().select(['B1', 'B2', 'B3'])
  tifToPNG(img, 'SR', aoi, filename=path+"new/"+filenamenew, scale=1.5, sclx=30)

  # 2010 version
  img = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate('2010-01', '2010-12-31').median().select(['B1', 'B2', 'B3'])
  tifToPNG(img, 'SR', aoi, filename=path+"2010/"+filename2010, scale=1.5, sclx=30)

Processing conversion for drive/My Drive/CV_DATA/2010/KushtiaBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/MaguraBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/MeherpurBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/NarailBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/SatkhiraBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/BograBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/NawabganjBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/joypurhatBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/NaogaonBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/PabnaBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/SirajganjBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/2010/DinajpurBANGLA2010.png...
Processing conversion for drive/My Drive/CV_DATA/

In [ ]:
DIR = 'drive/My Drive/CV_DATA/2010'
print (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

438
